<a href="https://colab.research.google.com/github/dfreire770/dq-ddq-smb-agent/blob/main/Mario_Test_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CODE COMES FROM HERE

https://www.kaggle.com/code/alincijov/super-mario-bros-double-deep-q-networks/notebook

In [ ]:
!pip install gym-super-mario-bros
!pip --disable-pip-version-check install -q nes_py
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip --disable-pip-version-check install -q pyvirtualdisplay

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already th

In [ ]:
import torch
from torch import nn
from torchvision import transforms as T

import time, datetime
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy

import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# change directory
import os
#os.chdir('/kaggle/input/gym-super-mario-bros'). <-- MAY NEED CHANGE HERE

# install controller
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

from pyvirtualdisplay import Display

from IPython import display as ipythondisplay
from IPython.display import HTML

#from gym.wrappers import Monitor -- DEPRECATED
from gym.wrappers.record_video import RecordVideo
from glob import glob

import base64
import io

display = Display(visible=0, size=(600, 300))
display.start()

In [ ]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render(mode='rgb_array')

env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which re

In [ ]:
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0")

# Limit the action-space to
#   0. walk right
#   1. jump right
env = JoypadSpace(env, [["right"], ["right", "A"]])

env.reset()
next_state, reward, done, info = env.step(action=0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


(240, 256, 3),
 0.0,
 False,
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}


In [ ]:
class SkipFrame(gym.Wrapper):
    '''
        Custom wrapper that inherits from gy.Wrapper and implements the step() function.
        Use it to return only every skip nth frame
    '''
    def __init__(self, env, skip):
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, info

In [ ]:
class ResizeObservation(gym.ObservationWrapper):
    '''
        Downsamples each observation into a square image.
        New size: [1, 84, 84]
    '''
    def __init__(self, env, shape):
        super().__init__(env)
        if isinstance(shape, int): self.shape = (shape, shape)
        else: self.shape = tuple(shape)

        obs_shape = self.shape + self.observation_space.shape[2:]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

    def observation(self, observation):
        transforms = T.Compose([T.Resize(self.shape), T.Normalize(0, 255)])
        return transforms(observation).squeeze(0)

class GrayScaleObservation(gym.ObservationWrapper):
    '''
        Common wrapper to transform an RGB image to grayscale; doing so reduces the size of the state representation without losing useful information.
        Now the size of each state: [1, 240, 256]
    '''
    def __init__(self, env):
        super().__init__(env)
        obs_shape = self.observation_space.shape[:2]
        self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)
        self.transform = T.Grayscale()

    def permute_orientation(self, observation):
        observation = np.transpose(observation, (2, 0, 1))
        return torch.tensor(observation.copy(), dtype=torch.float)

    def observation(self, observation):
        observation = self.permute_orientation(observation)
        return self.transform(observation)

In [ ]:
# preprocess environment
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
env = FrameStack(env, num_stack=4)

# record every 25th episode
#env = RecordVideo(env, './video', episode_trigger=lambda x: x % 25 == 0) <---- WHEN TRYING THIS, IT CRASHES

In [ ]:

class MarioNet(nn.Module):
    """
        input -> (conv2d + relu) x 3 -> flatten -> (dense + relu) x 2 -> output
    """
    def __init__(self, input_dim, output_dim):
        super().__init__()
        c, w, h = input_dim

        if h != 84: raise ValueError(f"Expecting input height: 84, got: {h}")
        if w != 84: raise ValueError(f"Expecting input width: 84, got: {w}")

        self.online = nn.Sequential(
            nn.Conv2d(in_channels=c, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )

        self.target = copy.deepcopy(self.online)

        # freeze Q-target parameters
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

In [ ]:
class Mario:
    '''
        Mario randomly explores with a chance of self.exploration_rate.
        When he chooses to exploit, he relies on MarioNet to provide the most optimal action.
    '''
    def __init__(self, state_dim, action_dim, use_cuda):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.use_cuda = use_cuda
        self.memory = deque(maxlen=100000)
        self.batch_size = 32

        self.net = MarioNet(self.state_dim, self.action_dim).float()
        if self.use_cuda:
            self.net = self.net.to(device="cuda")

        self.exploration_rate = 1
        self.exploration_rate_decay = 0.99999975
        self.exploration_rate_min = 0.1
        self.curr_step = 0

        self.save_every = 5e5


    def act(self, state):
        '''
            Given a state, choose an epsilon-greedy action and update value of step.
        '''
        # exploration
        if np.random.rand() < self.exploration_rate:
            action_idx = np.random.randint(self.action_dim)

        # exploitation
        else:
            state = state.__array__()

            if self.use_cuda: state = torch.tensor(state).cuda()
            else: state = torch.tensor(state)

            state = state.unsqueeze(0)
            action_values = self.net(state, model="online")
            action_idx = torch.argmax(action_values, axis=1).item()

        # decrease exploration_rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # increment step
        self.curr_step += 1
        return action_idx


    def cache(self, state, next_state, action, reward, done):
        """
            Store the experience to self.memory (replay buffer).
        """
        state = state.__array__()
        next_state = next_state.__array__()

        state = torch.tensor(state)
        next_state = torch.tensor(next_state)
        action = torch.tensor([action])
        reward = torch.tensor([reward])
        done = torch.tensor([done])

        if self.use_cuda:
            state = state.cuda()
            next_state = next_state.cuda()
            action = action.cuda()
            reward = reward.cuda()
            done = done.cuda()

        self.memory.append((state, next_state, action, reward, done,))


    def recall(self):
        """
            Retrieve a batch of experiences from memory
        """
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(torch.stack, zip(*batch))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

In [ ]:
class Mario(Mario):
    def __init__(self, state_dim, action_dim, use_cuda):
        super().__init__(state_dim, action_dim, use_cuda)
        self.gamma = 0.9
        self.burnin = 1e4  # min. experiences before training
        self.learn_every = 3  # no. of experiences between updates to Q_online
        self.sync_every = 1e4  # no. of experiences between Q_target & Q_online sync
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
        self.loss_fn = torch.nn.SmoothL1Loss()


    def td_estimate(self, state, action):
        current_Q = self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ]
        return current_Q


    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state_Q = self.net(next_state, model="online")
        best_action = torch.argmax(next_state_Q, axis=1)
        next_Q = self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()


    def update_Q_online(self, td_estimate, td_target):
        loss = self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()


    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())


    def learn(self):
        if self.curr_step % self.sync_every == 0: self.sync_Q_target()
        if self.curr_step % self.save_every == 0: self.save()
        if self.curr_step < self.burnin: return None, None
        if self.curr_step % self.learn_every != 0: return None, None

        # Sample from memory
        state, next_state, action, reward, done = self.recall()

        # Get TD Estimate
        td_est = self.td_estimate(state, action)

        # Get TD Target
        td_tgt = self.td_target(reward, next_state, done)

        # Backpropagate loss through Q_online
        loss = self.update_Q_online(td_est, td_tgt)

        return (td_est.mean().item(), loss)

In [ ]:

class MetricLogger:
    def __init__(self):
        # history metrics
        self.ep_rewards = []
        self.ep_lengths = []
        self.ep_avg_losses = []
        self.ep_avg_qs = []

        # moving averages, added for every call to record()
        self.moving_avg_ep_rewards = []
        self.moving_avg_ep_lengths = []
        self.moving_avg_ep_avg_losses = []
        self.moving_avg_ep_avg_qs = []

        # current episode metric
        self.init_episode()

        # timing
        self.record_time = time.time()

    def log_step(self, reward, loss, q):
        self.curr_ep_reward += reward
        self.curr_ep_length += 1
        if loss:
            self.curr_ep_loss += loss
            self.curr_ep_q += q
            self.curr_ep_loss_length += 1

    def log_episode(self):
        self.ep_rewards.append(self.curr_ep_reward)
        self.ep_lengths.append(self.curr_ep_length)
        if self.curr_ep_loss_length == 0:
            ep_avg_loss = 0
            ep_avg_q = 0
        else:
            ep_avg_loss = np.round(self.curr_ep_loss / self.curr_ep_loss_length, 5)
            ep_avg_q = np.round(self.curr_ep_q / self.curr_ep_loss_length, 5)
        self.ep_avg_losses.append(ep_avg_loss)
        self.ep_avg_qs.append(ep_avg_q)

        self.init_episode()

    def init_episode(self):
        self.curr_ep_reward = 0.0
        self.curr_ep_length = 0
        self.curr_ep_loss = 0.0
        self.curr_ep_q = 0.0
        self.curr_ep_loss_length = 0

    def record(self, episode, epsilon, step):
        mean_ep_reward = np.round(np.mean(self.ep_rewards[-100:]), 3)
        mean_ep_length = np.round(np.mean(self.ep_lengths[-100:]), 3)
        mean_ep_loss = np.round(np.mean(self.ep_avg_losses[-100:]), 3)
        mean_ep_q = np.round(np.mean(self.ep_avg_qs[-100:]), 3)
        self.moving_avg_ep_rewards.append(mean_ep_reward)
        self.moving_avg_ep_lengths.append(mean_ep_length)
        self.moving_avg_ep_avg_losses.append(mean_ep_loss)
        self.moving_avg_ep_avg_qs.append(mean_ep_q)

        last_record_time = self.record_time
        self.record_time = time.time()
        time_since_last_record = np.round(self.record_time - last_record_time, 3)

        print(
            "Episode:{:4d}  :: Step:{:5d}  :: Epsilon:{:8.3f}  :: Mean_Reward:{:8.3f}  :: " \
            "Mean_Length:{:8.3f}  :: Mean_Loss:{:4.3f}  :: Mean_Q_Value:{:8.3f}  :: " \
            "Time_Delta:{:8.3f} "
            .format(episode, step, epsilon, mean_ep_reward, mean_ep_length,
                    mean_ep_loss, mean_ep_q, time_since_last_record)
        )

In [ ]:

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

#path = 'https://github.com/dfreire770/dq-ddq-smb-agent/tree/main/'
path ='./'

def show_video():
    mp4list = glob(path+ '*.mp4')
    print(mp4list) # DEBUG
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env):
    #env = Monitor(env, path, force=True).  <--- Monitor is deprecated
    #env = RecordVideo(env, path)
    env = RecordVideo(env, './video', episode_trigger=lambda x: x % 25 == 0)
    return env


display = Display(visible=0, size=(600, 300))
display.start()


#env = wrap_env(env)

In [ ]:
'''
if torch.cuda.is_available():
    episodes = 301
else:
   episodes = 26

'''

use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
print()


mario = Mario(state_dim=(4, 84, 84), action_dim=env.action_space.n, use_cuda=use_cuda)

logger = MetricLogger()

#episodes = 26
episodes = 301


# record every 25th episode
#env = RecordVideo(env, './video', episode_trigger=lambda x: x % 25 == 0) # <---- WHEN TRYING THIS, IT CRASHES

# Start the recorder
#env.start_video_recorder()  #<----- START VIDEO RECORD

for e in range(episodes):
    state = env.reset()
    #env.render('rgb')

    # Play the game!
    while True:

        # Run agent on the state
        action = mario.act(state)

        # Agent performs action
        next_state, reward, done, info = env.step(action)

        #env.render('rgb_array') ### <-- ADDED HERE

        # Remember
        mario.cache(state, next_state, action, reward, done)

        # Learn
        q, loss = mario.learn()

        # Logging
        logger.log_step(reward, loss, q)

        # Update state
        state = next_state

        # Check if end of game
        if done or info["flag_get"]:
            break

    logger.log_episode()

    if e % 25 == 0:
        logger.record(episode=e, epsilon=mario.exploration_rate, step=mario.curr_step)
        #env.close_video_recorder() #<----- STOP VIDEO RECORD
        #env.close()
        show_video()

Using CUDA: False



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Episode:   0  :: Step:  163  :: Epsilon:   1.000  :: Mean_Reward: 635.000  :: Mean_Length: 163.000  :: Mean_Loss:0.000  :: Mean_Q_Value:   0.000  :: Time_Delta:   1.210 
[]
Could not find video
Episode:  25  :: Step: 4819  :: Epsilon:   0.999  :: Mean_Reward: 630.077  :: Mean_Length: 185.346  :: Mean_Loss:0.000  :: Mean_Q_Value:   0.000  :: Time_Delta:  35.087 
[]
Could not find video
Episode:  50  :: Step:10044  :: Epsilon:   0.997  :: Mean_Reward: 660.451  :: Mean_Length: 196.941  :: Mean_Loss:0.059  :: Mean_Q_Value:   0.036  :: Time_Delta:  39.805 
[]
Could not find video
Episode:  75  :: Step:13948  :: Epsilon:   0.997  :: Mean_Reward: 621.868  :: Mean_Length: 183.526  :: Mean_Loss:0.474  :: Mean_Q_Value:   1.154  :: Time_Delta:  61.012 
[]
Could not find video
Episode: 100  :: Step:20324  :: Epsilon:   0.995  :: Mean_Reward: 662.150  :: Mean_Length: 201.610  :: Mean_Loss:0.490  :: Mean_Q_Value:   1.782  :: Time_Delta: 101.172 
[]
Could not find video
Episode: 125  :: Step:24641  :

In [ ]:
show_video()

In [ ]:
state = env.reset()
env = wrap_env(env)

# Play the game!
while True:

    # Run agent on the state
    action = mario.act(state)

    # Agent performs action
    next_state, reward, done, info = env.step(action)

    # Update state
    state = next_state

    # Check if end of game
    if done or info["flag_get"]:
        break

#env.close()
show_video()

In [ ]:
#/content/https:/github.com/dfreire770/dq-ddq-smb-agent/tree/main/rl-video-episode-0.mp4